In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("text.csv")

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668440 entries, 0 to 668439
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ROW_ID  668440 non-null  int64 
 1   TEXT    668440 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [45]:
def truncate_text_to_512(text):
    # Ensure that text is a string
    if not isinstance(text, str):
        return text
    words = text.split()
    truncated = " ".join(words[:512])
    return truncated

# If clean_text is your Series of cleaned text observations, then:
df["TEXT"] = df["TEXT"].apply(truncate_text_to_512)

# To display the first few truncated texts:
print(df["TEXT"].head())

0    This is an 81-year-old female with a history o...
1    81 yo F smoker w/ COPD, severe TBM, s/p trache...
2    COPD exacerbation/Shortness of Breath. 87 yo F...
3    Mr. was seen at after a mechanical fall from a...
4    Meningioma. is a 62-year-old woman, with longs...
Name: TEXT, dtype: object


In [46]:
import re
import pandas as pd

# Define the vehicle regex (as provided)
vehicleRegex = (
    r'\b(?:mva|mba|vehicle|bus|pedestrian|passenger|ute|ped|bike|dirtbike|motorbike|pushbike|scooter|truck|'
    r'bicycle|motorcycle|driver|driving|rtc|rta|\d*km[a-zA-Z/]*|skateboard|surfing|surf|horse|collision|'
    r'crossing|buggy|ebike|jetski|vs car|car vs|car accident|moving car|traffic light|traffic lights|'
    r'hit by car|hit by a car|car hit|airbag|airbags|T boned)\b'
)

# Compile the regex with case-insensitive flag.
pattern = re.compile(vehicleRegex, re.IGNORECASE)

def predict_vehicle(text):
    """
    Given a text string, return a tuple (label, responsible_keywords) where:
      - label is True if any vehicle-related keyword is found, else False.
      - responsible_keywords is a comma-separated string of the matched keywords (or None if no match).
    """
    # Find all matching keywords in the text.
    matches = pattern.findall(text)
    if matches:
        # Remove duplicates by converting to a set, then back to a list.
        unique_matches = list(set(matches))
        # Optionally, you can sort the keywords.
        unique_matches.sort()
        return True, ", ".join(unique_matches)
    else:
        return False, None

# Now, assume you have a DataFrame named df with a column "TEXT"
# Apply the function to each row of df["TEXT"] and create new columns:
df[['vehicle_label', 'KEY']] = df['TEXT'].apply(lambda x: pd.Series(predict_vehicle(x)))

# For example, to inspect the first few rows:
print(df[['TEXT', 'vehicle_label', 'KEY']].head())


                                                TEXT  vehicle_label   KEY
0  This is an 81-year-old female with a history o...          False  None
1  81 yo F smoker w/ COPD, severe TBM, s/p trache...          False  None
2  COPD exacerbation/Shortness of Breath. 87 yo F...          False  None
3  Mr. was seen at after a mechanical fall from a...          False  None
4  Meningioma. is a 62-year-old woman, with longs...          False  None


In [47]:
df["vehicle_label"].value_counts()

vehicle_label
False    651625
True      16815
Name: count, dtype: int64

In [48]:
sub = df[df["vehicle_label"] == True]

In [49]:
sub

,ROW_ID,TEXT,vehicle_label,KEY
70,220,Pedestrian struck by motor vehicle. 60 year ol...,True,"Pedestrian, pedestrian, vehicle"
106,34,Seventy-eight-year-old female involved in a mo...,True,"driver, truck, vehicle"
107,35,HISTORY OF PRESENT ILLNESS: Seventy-eight-year...,True,"driver, truck, vehicle"
174,105,The patient is a 78-year-old male with a histo...,True,driver
226,159,"Sharp abdominal pain after cough, gross hematu...",True,MVA
...,...,...,...,...
667498,491227,"I saw and examined the patient, and was physic...",True,MVA
667506,491252,"I saw and examined the patient, and was physic...",True,MVA
668154,378803,"47 yo s/p MVC truck vs. pole, unrestrained dri...",True,"driver, truck"
668158,444123,29yM s/p pedestrian struck in MVC w/ bilateral...,True,pedestrian


In [54]:
value_counts = sub["KEY"].value_counts()

for key, count in value_counts.items():
    print(f"{key}: {count}")

VEHICLE: 4392
MVA: 1855
PEDESTRIAN: 843
motorcycle: 781
ped: 763
mva: 643
MOTORCYCLE: 435
driver: 380
driving: 377
pedestrian: 346
vehicle: 305
bike: 299
passenger: 284
MVA, VEHICLE: 233
bicycle: 222
driver, vehicle: 220
collision: 196
horse: 180
truck: 152
bus: 120
airbag, passenger: 111
PED: 108
PEDESTRIAN, ped: 93
passenger, vehicle: 81
pedestrian, vs car: 78
hit by car: 77
airbag: 74
car vs: 72
hit by a car: 62
pedestrian, vehicle: 62
driver, passenger: 61
scooter: 60
crossing: 59
VEHICLE, mva: 58
driver, truck: 54
MOTORCYCLE, motorcycle: 54
airbags, passenger: 54
collision, driver, vehicle: 53
vs car: 50
PEDESTRIAN, pedestrian: 49
car vs, driver: 46
collision, vehicle: 45
airbag, driver: 44
car accident: 43
VEHICLE, motorcycle: 42
BICYCLE: 39
MVA, driver: 36
ped, vs car: 35
bike, vs car: 35
MVA, pedestrian: 34
VEHICLE, driver: 32
MVA, passenger: 31
Pedestrian, pedestrian: 30
MVA, motorcycle: 30
motorcycle, vs car: 30
skateboard: 30
MVA, passenger, vehicle: 29
ped, vehicle: 29
COLL

In [41]:
sub["TEXT"].iloc[5]

'transfer from cath lab s/p cardiac arrest at OSH. 58 yo M with h/o HTN, hyperlipidemia, no known CAD, transferred from OSH s/p V-fib arrest AMI. Patient reports having chest pain, back pain under left scapula and left shoulder pain starting at 6 pm on while he was driving. He took 2 ASA at that time, the pain stopped then started again later on. Patient then went home and developed severe pain while at rest which did not resolve. He was driving to ED by his wife. At the OSH patient found to have ST elevations in V2-V6, went into V-fib, s/p two shock at 200J then 360 J down 7 min per flow sheet, patient started on lidocaine, nitro gtt\'s, integrilin and heparin started. He was A+O after this and transferred to for cath. Upon arrival patient still A+O, s/p successfull intervention with DES to mid LAD. Upon arrival to the CCU, patient pain free, A+O, integrilin running, off lidocaine. Having some AIVR on tele s/p 5 mg IV lopressor. Patient has never had similar pain in the past, is able 